In [1]:
!python3 --version

Python 3.6.9


In [2]:
from glob import glob
import os

In [3]:
# Tensorflow to ONNX conversion is supported through the tf2onnx converter.
import onnxmltools
import tensorflow as tf
import tf2onnx

To convert a TensorFlow model, tf2onnx prefers a frozen TensorFlow graph with specified inputs and outputs for the graph, alongside a source checkpoint file. More details on usage can be found in the [tf2onnx](https://github.com/onnx/tensorflow-onnx#usage) repository.

```
python3 -m tf2onnx.convert 
    --input SOURCE_GRAPHDEF_PB
    --graphdef SOURCE_GRAPHDEF_PB
    --checkpoint SOURCE_CHECKPOINT
    --saved-model SOURCE_SAVED_MODEL
    [--inputs GRAPH_INPUTS]
    [--outputs GRAPH_OUTPUS]
    [--inputs-as-nchw inputs_provided_as_nchw]
    [--target TARGET]
    [--output TARGET_ONNX_GRAPH]
    [--target TARGET]
    [--continue_on_error]
    [--verbose]
    [--custom-ops list-of-custom-ops]
    [--opset OPSET]
    [--fold_const]
```

In [102]:
models = sorted(glob("../trained_models/*"))
models

['../trained_models/began_origin',
 '../trained_models/builtin-image-classification-test-00',
 '../trained_models/builtin-image-classification-test-01',
 '../trained_models/custom-fmnist-resnet-test-00',
 '../trained_models/custom-fmnist-resnet-test-01',
 '../trained_models/custom-fmnist-test-01',
 '../trained_models/dcgan_origin',
 '../trained_models/embeddings']

In [6]:
import os

In [7]:
os.getcwd()

'/scripts/converter_scripts'

In [11]:
!pwd

/scripts/converter_scripts


# SavedModel

In [83]:
glob(f"{models[4]}/**/*")

['../trained_models/custom-fmnist-resnet-test-01/variables/variables.index',
 '../trained_models/custom-fmnist-resnet-test-01/variables/variables.data-00000-of-00001',
 '../trained_models/custom-fmnist-resnet-test-01/variables/variables_temp_c6910d302e31420bb82da528ba3cdd59',
 '../trained_models/custom-fmnist-resnet-test-01/variables/variables.data-00000-of-00002',
 '../trained_models/custom-fmnist-resnet-test-01/variables/variables.data-00001-of-00002',
 '../trained_models/custom-fmnist-resnet-test-01/logs/train',
 '../trained_models/custom-fmnist-resnet-test-01/logs/validation']

In [46]:
os.getcwd()

'/scripts/converter_scripts'

In [47]:
!saved_model_cli show \
    --dir "../trained_models/custom-fmnist-resnet-test-01" \
    --tag_set serve  \
    --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3)
      name: serving_default_input_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['activation_48'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 2048)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [101]:
# savedmodel is the name of the directory from your tf.saved_model output
# model.onnx is the name of your desired output model

!python3 -m tf2onnx.convert \
        --saved-model "../trained_models/custom-fmnist-resnet-test-01" \
        --output "../trained_models/custom-fmnist-resnet-test-01/model.onnx" \
        #--target rs6 \
        --fold_const \
        --verbose


2020-04-16 15:38:30,914 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/verbose_logging.py:72: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

2020-04-16 15:38:34,592 - INFO - Using tensorflow=1.15.2, onnx=1.6.0, tf2onnx=1.5.5/3c8f90
2020-04-16 15:38:34,592 - INFO - Using opset <onnx, 8>
2020-04-16 15:38:34,646 - ERROR - pass1 convert failed for name: "StatefulPartitionedCall"
op: "StatefulPartitionedCall"
input: "serving_default_input_1"
input: "conv1/kernel"
input: "conv1/bias"
input: "bn_conv1/gamma"
input: "bn_conv1/beta"
input: "bn_conv1/moving_mean"
input: "bn_conv1/moving_variance"
input: "res2a_branch2a/kernel"
input: "res2a_branch2a/bias"
input: "bn2a_branch2a/gamma"
input: "bn2a_branch2a/beta"
input: "bn2a_branch2a/moving_mean"
input: "bn2a_branch2a/moving_variance"
input: "res2a_branch2b/kernel"
input: "res2a_branch2b/bias"
input: "bn2a_branch2b/gamma"
input: "bn2a_branch2b/beta"
input: "bn2a_branch2b/

In [4]:
#!pip install git+https://github.com/microsoft/onnxconverter-common
#!pip install git+https://github.com/onnx/keras-onnx
    
os.environ["TF_KERAS"] = "1"  # Set before importing keras

import keras2onnx
import onnx
import onnxmltools
from keras.models import load_model

Using TensorFlow backend.


In [8]:
tfkeras_model = tf.keras.models.load_model("../trained_models/custom-fmnist-resnet-test-01")
onnx_model = keras2onnx.convert_keras(tfkeras_model, name="aa-tfkeras", target_opset=int(11))

The tf.op node resnet50_3/cond/Merge of type Merge cannot be converted
There is an error(<class 'AssertionError'>) happened during optimizing on the converted model!

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/keras2onnx/topology.py", line 317, in convert_topology
    target_opset=container.target_opset)
  File "/usr/local/lib/python3.6/dist-packages/onnxconverter_common/optimizer.py", line 1621, in optimize_onnx_graph
    initializers)
  File "/usr/local/lib/python3.6/dist-packages/onnxconverter_common/optimizer.py", line 302, in build_from_onnx
    assert var_ == '' or var_ in inputs
AssertionError

The maximum opset needed by this model is only 1.


In [ ]:
onnx.save_model(onnx_model, '../trained_models/custom-fmnist-resnet-test-01/model.onnx')

In [ ]:
# savedmodel is the name of the directory from your tf.saved_model output
# model.onnx is the name of your desired output model

!python3 -m tf2onnx.convert \
        --saved-model "../trained_models/custom-fmnist-resnet-test-01" \
        --output "../trained_models/custom-fmnist-resnet-test-01/model.onnx" \
        #--target rs6 \
        --fold_const \
        --verbose

# CKPT

In [49]:
!which bazel

/usr/local/bin/bazel


In [84]:
models

['../trained_models/began_origin',
 '../trained_models/builtin-image-classification-test-00',
 '../trained_models/builtin-image-classification-test-01',
 '../trained_models/custom-fmnist-resnet-test-00',
 '../trained_models/custom-fmnist-resnet-test-01',
 '../trained_models/custom-fmnist-test-01',
 '../trained_models/dcgan_origin',
 '../trained_models/embeddings']

In [92]:
!ls ../trained_models/**/*|grep pb

../trained_models/builtin-image-classification-test-00/graph.pbtxt
../trained_models/builtin-image-classification-test-01/graph.pbtxt
../trained_models/custom-fmnist-resnet-test-00/graph.pbtxt
../trained_models/custom-fmnist-resnet-test-01/saved_model.pb
../trained_models/custom-fmnist-test-01/graph.pbtxt


In [52]:
!which summarize_graph  input_layer

/usr/local/bin/summarize_graph


In [94]:
# !/scripts/tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
#   --in_graph="../trained_models/custom-fmnist-resnet-test-00/graph.pbtxt"
# =====
# Segmentation fault (core dumped)

!summarize_graph \
  --in_graph="../trained_models/custom-fmnist-resnet-test-00/graph.pbtxt"

No inputs spotted.
No variables spotted.
Found 1360 possible outputs: (name=global_step/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=global_step/add, op=AddV2) (name=MakeIterator, op=MakeIterator) (name=IteratorToStringHandle, op=IteratorToStringHandle) (name=conv1/kernel/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=conv1/kernel/Assign, op=AssignVariableOp) (name=conv1/bias/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=conv1/bias/Assign, op=AssignVariableOp) (name=bn_conv1/gamma/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=bn_conv1/gamma/Assign, op=AssignVariableOp) (name=bn_conv1/beta/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=bn_conv1/beta/Assign, op=AssignVariableOp) (name=bn_conv1/moving_mean/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=bn_conv1/moving_variance/IsInitialized/VarIsInitializedOp, op=VarIsInitializedOp) (name=bn_conv1/AssignMovingAvg/ReadVariableOp_1, op=ReadVa

In [99]:
!saved_model_cli show \
    --dir "../trained_models/custom-fmnist-resnet-test-01" \
    --tag_set serve  \
    --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3)
      name: serving_default_input_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['activation_48'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 2048)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [100]:
# savedmodel is the name of the directory from your tf.saved_model output
# model.onnx is the name of your desired output model

!python3 -m tf2onnx.convert \
        --checkpoint "../trained_models/custom-fmnist-resnet-test-00/model.ckpt-93757.meta" \
        --inputs "serving_default_input_1:0" \
        --outputs "StatefulPartitionedCall:0" \
        --output "/scripts/aa/custom-fmnist-test-00/model.onnx" \
        --target rs6 \
        --fold_const \
        --verbose


2020-04-16 15:34:36,950 - WARNING - tensorflow: From /usr/local/lib/python3.6/dist-packages/tf2onnx/verbose_logging.py:72: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


2020-04-16 15:34:36,950 - WARNING - tensorflow: From /usr/local/lib/python3.6/dist-packages/tf2onnx/loader.py:74: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2020-04-16 15:34:36,950 - WARNING - tensorflow: From /usr/local/lib/python3.6/dist-packages/tf2onnx/loader.py:76: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.


2020-04-16 15:34:39,602 - WARNING - tensorflow: From /usr/local/lib/python3.6/dist-packages/tf2onnx/loader.py:77: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-04-16 15:34:39.604046: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary

https://github.com/tensorflow/tensorflow/issues/31318: Cannot assign a device
Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint.

This is fixed with tf-nightly version '2.2.0-dev20200303'. Thanks!

In [18]:
!pip3 install keras2onnx

In [19]:
# savedmodel is the name of the directory from your tf.saved_model output
# model.onnx is the name of your desired output model

!python3 -m keras2onnx.convert \
        --saved-model "/scripts/aa/test-01/" \
        --output "/scripts/aa/test-01/model.onnx" \
        --target rs6 \
        --fold_const \
        --verbose

Using TensorFlow backend.
/usr/bin/python3: No module named keras2onnx.convert


In [ ]:
model = tf.keras.models.load_model(my_model_folder_path)
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, model_name_onnx)

In [ ]:
# savedmodel is the name of the directory from your tf.saved_model output
# model.onnx is the name of your desired output model

!python3 -m tf2onnx.convert \
        --saved-model savedmodel \
        --output model.onnx \
        --target rs6 \
        --fold_const \
        --verbose

In [ ]:
# model.pb is the name of your checkpoint
# graph_inputs are the names of your input nodes
# graph_outputs are the names of your output nodes 
# model.onnx is the name of your desired output model

# If you know your graph input / output node names, then you can use the freeze_graph method instead
# input and output node names can be discovered through Netron, tensorboard, or the TF summarize_graph tool

!python3 -m tf2onnx.convert \
        --input model.pb \
        --inputs GRAPH_INPUTS \
        --outputs GRAPH_OUTPUTS \
        --output model.onnx \
        --fold_const \
        --verbose

Alternatively, you could use a model from an active Tensorflow session in Python, freeze the graph, and then convert it, as demonstrated below.

In [ ]:
# Replace this with your desired input TF model name
input_model_name = "tf_model"

# Replace this with your desired output ONNX model name
output_onnx_model = "model.onnx"

In [ ]:
with tf.Session() as sess:
    # Note: this is a simple example Tensorflow model
    x = tf.placeholder(tf.float32, [2, 3], name="input")
    x_ = tf.add(x, x)
    _ = tf.identity(x_, name="output")
    onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph, input_names=["input:0"], output_names=["output:0"])
    onnx_model = onnx_graph.make_model(input_model_name)

In [ ]:
# Save as protobuf

onnxmltools.utils.save_model(onnx_model, output_onnx_model)